In [1]:
## import (and install if necessary) pandas
%pip install pandas
import pandas as pd
from datetime import datetime


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
df = pd.read_csv("full_data.csv"
                 ,parse_dates=['TripDate','StartDateTime','EndDateTime','CustomerBirthDate']
                 )
## process cost monetary data
df['Cost'] = df['Cost'].str.replace(',', '')
df['Cost'] = df['Cost'].str.replace('$', '')
df['Cost'] = df['Cost'].str.replace('(', '')
df['Cost'] = df['Cost'].str.replace(')', '')
df['Cost'] = df['Cost'].astype(float)
print(df.info())
print(df.head(5))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1160755 entries, 0 to 1160754
Data columns (total 14 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   CustomerID         1160755 non-null  int64         
 1   CustomerSex        1160755 non-null  object        
 2   CustomerBirthDate  1160417 non-null  datetime64[ns]
 3   TripDate           1160755 non-null  datetime64[ns]
 4   VehicleID          1160755 non-null  int64         
 5   RunActualID        1160755 non-null  int64         
 6   StartDateTime      1160755 non-null  datetime64[ns]
 7   EndDateTime        1160755 non-null  datetime64[ns]
 8   PickupX            1110755 non-null  float64       
 9   PickupY            1110755 non-null  float64       
 10  DropoffX           1109028 non-null  float64       
 11  DropoffY           1109028 non-null  float64       
 12  Cost               1160538 non-null  float64       
 13  CollectedFareType  1147941 

In [19]:
## summary counts
runActualID_counts = df.groupby(['RunActualID'])['RunActualID'].count()
print(f"Maximum Number of rows per RunActualID: {runActualID_counts.max()}")
runActualID_vehicleID = df.groupby('RunActualID')['VehicleID'].nunique()
runActualIDs_more_than_one_vehicleID = runActualID_vehicleID[runActualID_vehicleID != 1]
print(f"Any RunActualIDs that have more than one VehicleID: {runActualIDs_more_than_one_vehicleID.nunique()}")
print(f"Unique number of RunActualIDs: {df['RunActualID'].nunique()}")
print(f"Unique number of CustomerIDs: {df['CustomerID'].nunique()}")
print(f"Unique number of VehicleIDs: {df['VehicleID'].nunique()}")

Maximum Number of rows per RunActualID: 18
Any RunActualIDs that have more than one VehicleID: 0
Unique number of RunActualIDs: 213272
Unique number of CustomerIDs: 10215
Unique number of VehicleIDs: 538


In [4]:
dates_per_runactualIDs = df.groupby(['RunActualID']).agg({'EndDateTime': 'max', 'StartDateTime':'min'})
dates_per_runactualIDs['EndDate'] = pd.to_datetime(dates_per_runactualIDs['EndDateTime'].dt.date)
dates_per_runactualIDs['StartDate'] = pd.to_datetime(dates_per_runactualIDs['StartDateTime'].dt.date)
dates_per_runactualIDs['diff'] = (dates_per_runactualIDs['EndDateTime'] - dates_per_runactualIDs['StartDateTime']).dt.total_seconds()
seconds_in_day = 86400
runactualIDs_over_day = dates_per_runactualIDs[dates_per_runactualIDs['diff'] > seconds_in_day]
#print(runactualIDs_over_day['RunActualId'].nunique())

print(f"""Number of RunActualIDs lasting over 24 hours: {runactualIDs_over_day.index.nunique()}
(Calculated by subtracting EndDateTime from StartDateTime)
Saving outlier RunActualIDs to `runactualIDs_over_day.csv""")
runactualIDs_over_day.to_csv("runactualIDs_over_day.csv", columns = ['StartDateTime','EndDateTime'])

Number of RunActualIDs lasting over 24 hours: 4
(Calculated by subtracting EndDateTime from StartDateTime)
Saving outlier RunActualIDs to `runactualIDs_over_day.csv


         CustomerID CustomerSex CustomerBirthDate   TripDate  VehicleID  \
0             68809           M        1972-10-02 2019-01-01       1832   
1             56045           F        1957-10-25 2019-01-01       1803   
2             54784           F        1965-06-04 2019-01-01       1938   
3            126788           F        1966-01-22 2019-01-01       1765   
4            104367           F        1948-11-15 2019-01-01       1614   
...             ...         ...               ...        ...        ...   
1160750       94977           F        1960-09-23 2021-06-30       1803   
1160751       57195           F        1941-03-23 2021-06-30       1936   
1160752       92775           F        1956-08-22 2021-06-30       1803   
1160753       81412           F        1951-03-21 2021-06-30       1936   
1160754      120015           F        1962-08-01 2021-06-30       1803   

         RunActualID       StartDateTime         EndDateTime     PickupX  \
0            2470597 20